# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

BASE_URL = "http://books.toscrape.com/"

RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5,
}

def scrape_books(min_rating: float, max_price: float) -> pd.DataFrame:
    books_data = []

    for page in range(1, 51):
        list_url = urljoin(BASE_URL, f"catalogue/page-{page}.html")
        print("Fetching list page:", list_url)

        resp = requests.get(list_url)
        if resp.status_code != 200:
            print("status_code != 200, stop at page", page)
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        product_list = soup.find_all("article", class_="product_pod")

        if not product_list:
            print("No products on this page, stop at page", page)
            break

        for prod in product_list:
            a_tag = prod.h3.a
            title = a_tag["title"].strip()
            detail_href = a_tag["href"]
            detail_url = urljoin(list_url, detail_href)

            price_text = prod.find("p", class_="price_color").get_text(strip=True)
            price_str = re.sub(r"[^0-9.]", "", price_text)
            if not price_str:
                continue
            price = float(price_str)

            rating_tag = prod.find("p", class_="star-rating")
            rating_classes = rating_tag.get("class", []) if rating_tag else []
            rating_word = None
            for c in rating_classes:
                if c in RATING_MAP:
                    rating_word = c
                    break
            rating = RATING_MAP.get(rating_word, None)

            if rating is None or rating < min_rating or price > max_price:
                continue

            detail_resp = requests.get(detail_url)
            if detail_resp.status_code != 200:
                continue
            detail_soup = BeautifulSoup(detail_resp.text, "html.parser")

            upc = None
            table = detail_soup.find("table", class_="table table-striped")
            if table:
                first_row = table.find("tr")
                if first_row:
                    upc = first_row.find("td").get_text(strip=True)

            genre = None
            breadcrumb = detail_soup.find("ul", class_="breadcrumb")
            if breadcrumb:
                li_tags = breadcrumb.find_all("li")
                if len(li_tags) >= 3:
                    genre = li_tags[-2].get_text(strip=True)

            availability = None
            avail_tag = detail_soup.find("p", class_="instock availability")
            if avail_tag:
                availability = avail_tag.get_text(strip=True)

            description = None
            desc_header = detail_soup.find(id="product_description")
            if desc_header:
                desc_p = desc_header.find_next_sibling("p")
                if desc_p:
                    description = desc_p.get_text(strip=True)

            books_data.append({
                "UPC": upc,
                "Title": title,
                "Price (£)": price,
                "Rating": rating,
                "Genre": genre,
                "Availability": availability,
                "Description": description,
            })

    print("Collected", len(books_data), "books in total") 
    return pd.DataFrame(books_data)

In [2]:
print("cell started")
df = scrape_books(min_rating=4.0, max_price=20.0)
print(df.head())
print("Total: ", len(df), "books")

cell started
Fetching list page: http://books.toscrape.com/catalogue/page-1.html
Fetching list page: http://books.toscrape.com/catalogue/page-2.html
Fetching list page: http://books.toscrape.com/catalogue/page-3.html
Fetching list page: http://books.toscrape.com/catalogue/page-4.html
Fetching list page: http://books.toscrape.com/catalogue/page-5.html
Fetching list page: http://books.toscrape.com/catalogue/page-6.html
Fetching list page: http://books.toscrape.com/catalogue/page-7.html
Fetching list page: http://books.toscrape.com/catalogue/page-8.html
Fetching list page: http://books.toscrape.com/catalogue/page-9.html
Fetching list page: http://books.toscrape.com/catalogue/page-10.html
Fetching list page: http://books.toscrape.com/catalogue/page-11.html
Fetching list page: http://books.toscrape.com/catalogue/page-12.html
Fetching list page: http://books.toscrape.com/catalogue/page-13.html
Fetching list page: http://books.toscrape.com/catalogue/page-14.html
Fetching list page: http://boo